# Trabalho Final - INF 493

> Alunos: Cleber Luiz Oliveira Junior, Vinicius Kuster Lodi


# Pré-Processamento

## Imports iniciais e upload da tabela

In [ ]:
# Imports iniciais:

import pandas as pd
import matplotlib as mtplt
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

In [ ]:
%pip install catboost xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

In [ ]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [ ]:
# Utilizando base_sem_mod_iniciais.xlsx
# Utilizando o Drive

drive.mount('/content/drive')
base = pd.read_excel('/content/drive/My Drive/INF 493 - Ciência de dados/TrabalhoFinal - INF 493/base_sem_as_mod_iniciais.xlsx')

Mounted at /content/drive


In [ ]:
base.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ANO_NOT,17905.0,NaN,NaN,NaN,2013.877297,5.464144,2000.0,2009.0,2017.0,2018.0,2018.0
UF_NOTIF,17905.0,NaN,NaN,NaN,34.150963,8.218683,9.0,31.0,34.0,35.0,99.0
REG_NOTIF,17905.0,NaN,NaN,NaN,3.124379,0.857144,1.0,3.0,3.0,3.0,9.0
IDADE,17905.0,NaN,NaN,NaN,37.333296,18.558619,0.001598,23.0,37.0,50.0,97.0
IDADE_CAT,17905.0,NaN,NaN,NaN,2.903826,0.707359,1.0,3.0,3.0,3.0,4.0
SEXO,17905.0,NaN,NaN,NaN,0.69327,0.506848,0.0,0.0,1.0,1.0,9.0
VACINA_FA,17905.0,NaN,NaN,NaN,3.386708,3.23689,1.0,1.0,2.0,2.0,9.0
SINT_HEMORRAGICO,17905.0,NaN,NaN,NaN,4.067635,3.364453,1.0,2.0,2.0,9.0,9.0
DISTUR_RENAL,17905.0,NaN,NaN,NaN,4.235074,3.429713,1.0,2.0,2.0,9.0,9.0
SINAL_FAGET,17905.0,NaN,NaN,NaN,4.378051,3.472395,1.0,2.0,2.0,9.0,9.0


## Observações

Observações:
- Sexo dividido em 0 e 1 (qual é qual?)

    - 0: feminino, 1: masculino

- O max de Sexo é 9, deve ter valores errados na tabela

    - Existem somente 11 aparições que não são 0 ou 1, todas do número 9, pode excluir as linhas ou colocar NULL e não considerar o sexo

- Coluna idade_cat? (oq significa?, provavelmente so excluir)

  - Categorias= idade : cat, 0-9 : 1, 10-19 : 2, 20-59 : 3, 60-99 : 4 / realmente reduntante


- Não agrega no modelo ter data_notificação e data_sintoma e data_internação, faz mais sentido criar dias_até_notificação e dias_até_internação (melhor para o modelo entender e relacionar)

    - Criado dias_ate_notif

- Criar coluna óbito (0/1), não precisa guardar a data do óbito (ajuda nos valores ausentes)

    - Feito

- Coluna data_encerramento parece inútil

    - realmente é

- Coluna peri_sazional? oq quer dizer os valores?

    - Peri_sazonal= perido : categoria, jan - abril : 1, maio - agosto : 3, setembro - dezembro : 2

- Existem idades entre 0 e 1 (n faz sentido)

    - Existem 590 instâncias onde o valor não é um inteiro (tipo esperado para idade), 590 não é um número tão significante dadas as quase 18000 instâncias, mas deve haver um jeito de tratar isso
    - Com o arredondamento dos valores para o inteiro mais próximo, a qtd de 0 e 1 se mostraram normais, logo o arredondamento se mostrou eficiente

- O max de mes_sintoma é 99, sugere valores errados (deveria ser 12)

    - Existem 23 aparições de números fora do intervalo 1-12, todos do número 99, podem ser removidas as linhas, NULL pode ser considerado tbm

- Aparentemente precisa olhar data_sintoma e data_notificação, parece que algumas DS estão depois de DN (não faz sentido), ou um intervalo muito grande (sugere erro)

    - Foi criada uma coluna com dias_ate_notif = data_notificacao - data_sintoma, essa coluna possui 46 valores negativos (não faz sentido) e 45 valores acima de 180, oq sugere que pessoas sentiram um sintoma e notificaram somente 6 meses depois (não faz sentido), sugiro remoção

Fazer:
- Ler a dissertação da karen pra ver o significado de cada coluna e ver se os valores fazem sentido
- Analisar colunas que sugerem valores errados para ver se são outliers ou só errado, e ver se remove a linha ou tenta modificar

##Coluna MES_SINTOMA

In [ ]:
print(base['MES_SINTOMA'].value_counts())


MES_SINTOMA
1     5636
2     3524
3     2878
4     1496
12     868
5      771
6      514
11     492
10     463
8      433
7      431
9      376
99      23
Name: count, dtype: int64


### Drop de mês 99

In [ ]:
# Remove linhas onde MES_SINTOMA é = 99
initial_rows = base.shape[0]
base = base[~((base['MES_SINTOMA'] == 99))]

removed_rows = initial_rows - base.shape[0]
print(f"Número de linhas removidas devido a MES_SINTOMA = 99: {removed_rows}")
print(f"Número de linhas após a remoção: {base.shape[0]}")

Número de linhas removidas devido a MES_SINTOMA = 99: 23
Número de linhas após a remoção: 17882


##Coluna SEXO

In [ ]:
print(base['SEXO'].value_counts())


SEXO
1    12300
0     5571
9       11
Name: count, dtype: int64


In [ ]:
# Remove linhas onde SEXO é = 9
initial_rows = base.shape[0]
base = base[~((base['SEXO'] == 9))]

removed_rows = initial_rows - base.shape[0]
print(f"Número de linhas removidas devido a SEXO = 9: {removed_rows}")
print(f"Número de linhas após a remoção: {base.shape[0]}")

Número de linhas removidas devido a SEXO = 9: 11
Número de linhas após a remoção: 17871


##Coluna IDADE

In [ ]:
print(base['IDADE'].value_counts().to_string())

IDADE
40.000000    372
36.000000    370
38.000000    365
30.000000    351
34.000000    351
35.000000    349
28.000000    349
37.000000    345
21.000000    339
19.000000    339
24.000000    338
42.000000    337
32.000000    327
27.000000    323
39.000000    322
33.000000    321
31.000000    320
25.000000    318
41.000000    316
26.000000    313
43.000000    312
20.000000    308
47.000000    307
46.000000    301
22.000000    300
23.000000    300
44.000000    293
50.000000    289
48.000000    288
29.000000    286
45.000000    283
18.000000    277
52.000000    272
49.000000    270
17.000000    266
54.000000    257
16.000000    246
53.000000    242
51.000000    238
59.000000    235
55.000000    230
56.000000    220
15.000000    216
57.000000    204
58.000000    198
60.000000    183
63.000000    177
14.000000    176
61.000000    176
62.000000    171
13.000000    155
64.000000    150
12.000000    144
65.000000    143
11.000000    133
67.000000    125
1.000000     124
66.000000    123
7.000000

In [ ]:
non_integer_age_count = base['IDADE'][base['IDADE'] != base['IDADE'].astype(int)].count()
print(f"Número de valores não inteiros na coluna 'IDADE': {non_integer_age_count}")

Número de valores não inteiros na coluna 'IDADE': 588


In [ ]:
base['IDADE'] = base['IDADE'].round().astype(int)
print(base['IDADE'].value_counts().to_string())

IDADE
36    381
40    378
38    371
35    362
30    358
34    357
37    356
28    354
19    345
24    344
21    343
42    342
27    329
33    329
32    329
31    327
39    327
25    323
41    322
26    319
43    319
20    315
47    314
46    311
23    305
22    303
44    300
50    299
48    295
29    291
45    290
52    286
18    283
49    274
17    269
54    263
53    250
16    249
51    246
59    245
55    236
56    223
15    221
57    211
1     203
58    199
60    188
61    183
63    179
14    178
62    176
13    160
64    155
65    151
12    147
11    134
0     130
67    128
66    127
7     119
8     117
9     113
10    105
4     104
6     103
70     99
2      96
69     94
68     94
5      87
71     86
3      81
72     78
73     63
74     53
75     51
79     49
78     45
77     44
76     42
80     31
83     26
82     25
81     25
86     18
84     18
85     12
88     11
87     10
90      8
93      8
91      7
89      6
94      5
92      3
95      2
97      1


##DIAS_ATE_NOTIF

In [ ]:
base['DATA_NOTIF'] = pd.to_datetime(base['DATA_NOTIF'], format='mixed', dayfirst=True, errors='coerce')
base['DATA_SINTOMA'] = pd.to_datetime(base['DATA_SINTOMA'], format='mixed', dayfirst=True, errors='coerce')
base['DIAS_ATE_NOTIF'] = (base['DATA_NOTIF'] - base['DATA_SINTOMA']).dt.days

display(base[['DATA_NOTIF', 'DATA_SINTOMA', 'DIAS_ATE_NOTIF']])

,DATA_NOTIF,DATA_SINTOMA,DIAS_ATE_NOTIF
0,2016-01-07,2016-01-05,2.0
1,2016-01-07,2016-01-05,2.0
2,2016-01-11,2016-01-05,6.0
3,2016-01-12,2016-01-08,4.0
4,2016-01-14,2016-01-13,1.0
...,...,...,...
17900,2001-04-17,2001-04-17,0.0
17901,2001-04-10,2001-04-04,6.0
17902,2001-09-17,2001-08-25,23.0
17903,2001-08-13,2001-08-04,9.0


In [ ]:
negative_days_df = base[base['DIAS_ATE_NOTIF'] < 0]
maiores_que_180_dias_df = base[base['DIAS_ATE_NOTIF'] > 180]

print(f"Número de instâncias onde DIAS_ATE_NOTIF é negativo: {negative_days_df.shape[0]}")
print("Instâncias onde DIAS_ATE_NOTIF é negativo:")
display(negative_days_df[['DATA_NOTIF', 'DATA_SINTOMA', 'DIAS_ATE_NOTIF']])

print(f"\nNúmero de instâncias onde DIAS_ATE_NOTIF é maior que 180: {maiores_que_180_dias_df.shape[0]}")
print("Instâncias onde DIAS_ATE_NOTIF é maior que 180:")
display(maiores_que_180_dias_df[['DATA_NOTIF', 'DATA_SINTOMA', 'DIAS_ATE_NOTIF']])

Número de instâncias onde DIAS_ATE_NOTIF é negativo: 46
Instâncias onde DIAS_ATE_NOTIF é negativo:


,DATA_NOTIF,DATA_SINTOMA,DIAS_ATE_NOTIF
6402,2017-01-08,2017-01-09,-1.0
6530,2017-02-24,2017-02-25,-1.0
6657,2017-01-20,2017-01-22,-2.0
6667,2017-01-25,2017-01-26,-1.0
6716,2017-01-25,2017-04-11,-76.0
6797,2017-01-23,2017-01-28,-5.0
7216,2017-01-12,2017-01-14,-2.0
7379,2017-01-19,2017-01-21,-2.0
7505,2017-01-23,2017-01-25,-2.0
7580,2017-01-25,2017-02-04,-10.0



Número de instâncias onde DIAS_ATE_NOTIF é maior que 180: 45
Instâncias onde DIAS_ATE_NOTIF é maior que 180:


,DATA_NOTIF,DATA_SINTOMA,DIAS_ATE_NOTIF
218,2016-10-21,2016-03-10 00:00:00.000000000,225.0
306,2017-01-09,2016-01-31 00:00:00.000000000,344.0
336,2017-01-16,2016-01-01 00:00:00.000000000,381.0
338,2017-01-17,2016-01-11 00:00:00.000000000,372.0
348,2017-01-20,2016-01-15 00:00:00.000000000,371.0
349,2017-01-20,2016-01-19 00:00:00.000000000,367.0
359,2017-01-29,2016-01-25 00:00:00.000000000,370.0
367,2017-02-09,2016-01-09 00:00:00.000000000,397.0
368,2017-02-10,2016-07-06 00:00:00.000000000,219.0
370,2017-03-09,2016-02-25 00:00:00.000000000,378.0


### Remoção de datas negativas e datas muito grandes

> Dias negativos são retirados e consideramos >100 dias como ruídos

In [ ]:
# Remove linhas onde DIAS_ATE_NOTIF é negativo ou maior que 100
initial_rows = base.shape[0]
base = base[~((base['DIAS_ATE_NOTIF'] < 0) | (base['DIAS_ATE_NOTIF'] > 100))]

removed_rows = initial_rows - base.shape[0]
print(f"Número de linhas removidas devido a DIAS_ATE_NOTIF negativo ou > 100: {removed_rows}")
print(f"Número de linhas após a remoção: {base.shape[0]}")

Número de linhas removidas devido a DIAS_ATE_NOTIF negativo ou > 100: 170
Número de linhas após a remoção: 17701


## REDUÇÃO DE LINHAS DUPLICADAS

In [ ]:
duplicate_rows = base[base.duplicated()]
print(f"Número de linhas duplicadas: {duplicate_rows.shape[0]}")

Número de linhas duplicadas: 24


In [ ]:
base.drop_duplicates(inplace=True)
print(f"Número de linhas após a remoção de duplicatas: {base.shape[0]}")

Número de linhas após a remoção de duplicatas: 17677


## CRIAÇÃO DA COLUNA ÓBITO


> 0: Não internado

> 1: Internado

In [ ]:
# Criar a coluna 'OBITO': 1 se DATA_OBITO não for nula, 0 se for nula
base['OBITO'] = base['DATA_OBITO'].notna().astype(int)

# Remover a coluna original 'DATA_OBITO'
base = base.drop('DATA_OBITO', axis=1, errors='ignore')

print("Coluna 'OBITO' criada com sucesso e 'DATA_OBITO' removida.")
print("Valores únicos na nova coluna 'OBITO':")
print(base['OBITO'].value_counts())

Coluna 'OBITO' criada com sucesso e 'DATA_OBITO' removida.
Valores únicos na nova coluna 'OBITO':
OBITO
0    15560
1     2117
Name: count, dtype: int64


In [ ]:
print(base['OBITO'].value_counts())

OBITO
0    15560
1     2117
Name: count, dtype: int64


## CRIAÇÃO DA COLUNA DIAS_ATE_INTERNAÇÃO

In [ ]:
# Criar coluna DIAS_ATE_INTERNACAO
base['DATA_INTERNACAO'] = pd.to_datetime(base['DATA_INTERNACAO'], format='mixed', dayfirst=True, errors='coerce')
base['DIAS_ATE_INTERNACAO'] = (base['DATA_INTERNACAO'] - base['DATA_SINTOMA']).dt.days
print("Coluna 'DIAS_ATE_INTERNACAO' criada com sucesso.")

print("\nColunas restantes no DataFrame:")
print(base.columns.tolist())

Coluna 'DIAS_ATE_INTERNACAO' criada com sucesso.

Colunas restantes no DataFrame:
['ANO_NOT', 'UF_NOTIF', 'REG_NOTIF', 'IDADE', 'IDADE_CAT', 'SEXO', 'VACINA_FA', 'SINT_HEMORRAGICO', 'DISTUR_RENAL', 'SINAL_FAGET', 'DOR_ABDOMINAL', 'HOSPITALIZACAO', 'RESULT_IGM', 'RESULT_PCR', 'HISTOPATOLOGIA', 'IMUNOHISTOQUIMICA', 'R_ISOVIRAL', 'CLAS_FINAL', 'CRIT_CONF', 'EVOLUCAO', 'AUTOCTONE', 'DATA_NOTIF', 'DATA_SINTOMA', 'DATA_INTERNACAO', 'DATA_ENCERRAMENTO', 'MES_SINTOMA', 'PERI_SAZONAL', 'DIAS_ATE_NOTIF', 'OBITO', 'DIAS_ATE_INTERNACAO']


In [ ]:
# Remove linhas onde DIAS_ATE_INTERNACAO é negativo
initial_rows = base.shape[0]
base = base[~((base['DIAS_ATE_INTERNACAO'] < 0))]

removed_rows = initial_rows - base.shape[0]
print(f"Número de linhas removidas devido a DIAS_ATE_INTERNACAO negativo: {removed_rows}")
print(f"Número de linhas após a remoção: {base.shape[0]}")

Número de linhas removidas devido a DIAS_ATE_INTERNACAO negativo: 151
Número de linhas após a remoção: 17526


## CRIAÇÃO DA COLUNA INTERNAÇÃO


> 0: Não internado

> 1: Internado

In [ ]:
# Criar a coluna 'INTERNACAO': 1 se DATA_INTERNACAO não for nula, 0 se for nula
base['INTERNACAO'] = base['DATA_INTERNACAO'].notna().astype(int)

print("Coluna 'INTERNACAO' criada com sucesso")
print("Valores únicos na nova coluna 'INTERNACAO':")
print(base['INTERNACAO'].value_counts())

Coluna 'INTERNACAO' criada com sucesso
Valores únicos na nova coluna 'INTERNACAO':
INTERNACAO
0    10027
1     7499
Name: count, dtype: int64


## REMOÇÃO DE COLUNAS

In [ ]:
# Remover colunas 'IDADE_CAT' e 'DATA_ENCERRAMENTO'
base = base.drop(['IDADE_CAT', 'DATA_ENCERRAMENTO'], axis=1, errors='ignore')
print("Colunas 'IDADE_CAT' e 'DATA_ENCERRAMENTO' removidas com sucesso (se existiam).")

# Remover colunas 'DATA_NOTIF, DATA_SINTOMA, DATA_INTERNACAO, DATA_ENCERRAMENTO, MES_SINTOMA'
base = base.drop(['DATA_NOTIF', 'DATA_SINTOMA', 'DATA_INTERNACAO', 'DATA_ENCERRAMENTO', 'MES_SINTOMA', 'ANO_NOT'], axis=1, errors='ignore')
print("Colunas 'DATA_NOTIF, DATA_SINTOMA, DATA_INTERNACAO, DATA_ENCERRAMENTO, MES_SINTOMA, ANO_NOT' removidas com sucesso (se existiam).")

Colunas 'IDADE_CAT' e 'DATA_ENCERRAMENTO' removidas com sucesso (se existiam).
Colunas 'DATA_NOTIF, DATA_SINTOMA, DATA_INTERNACAO, DATA_ENCERRAMENTO, MES_SINTOMA, ANO_NOT' removidas com sucesso (se existiam).


In [ ]:
print("\nColunas restantes no DataFrame:")
print(base.columns.tolist())


Colunas restantes no DataFrame:
['UF_NOTIF', 'REG_NOTIF', 'IDADE', 'SEXO', 'VACINA_FA', 'SINT_HEMORRAGICO', 'DISTUR_RENAL', 'SINAL_FAGET', 'DOR_ABDOMINAL', 'HOSPITALIZACAO', 'RESULT_IGM', 'RESULT_PCR', 'HISTOPATOLOGIA', 'IMUNOHISTOQUIMICA', 'R_ISOVIRAL', 'CLAS_FINAL', 'CRIT_CONF', 'EVOLUCAO', 'AUTOCTONE', 'PERI_SAZONAL', 'DIAS_ATE_NOTIF', 'OBITO', 'DIAS_ATE_INTERNACAO', 'INTERNACAO']


## ANÁLISE DAS COLUNAS...

In [ ]:
columns_to_process = ['SEXO', 'VACINA_FA', 'SINT_HEMORRAGICO', 'DISTUR_RENAL', 'SINAL_FAGET', 'DOR_ABDOMINAL', 'RESULT_IGM', 'RESULT_PCR', 'HISTOPATOLOGIA', 'IMUNOHISTOQUIMICA', 'R_ISOVIRAL', 'CLAS_FINAL', 'CRIT_CONF', 'EVOLUCAO']

for col in columns_to_process:
    if col in base.columns:
        print(f"\nProcessing column: {col}")
        initial_values = base[col].value_counts().to_dict()
        print(f"Initial value counts:\n{initial_values}")
        base[col] = base[col].replace([4, 9], 3)
        print(f"Updated value counts:\n{base[col].value_counts().to_dict()}")
    else:
        print(f"Column {col} not found in DataFrame, skipping.")


Processing column: SEXO
Initial value counts:
{1: 12047, 0: 5479}
Updated value counts:
{1: 12047, 0: 5479}

Processing column: VACINA_FA
Initial value counts:
{2: 7336, 1: 5899, 9: 4291}
Updated value counts:
{2: 7336, 1: 5899, 3: 4291}

Processing column: SINT_HEMORRAGICO
Initial value counts:
{2: 9543, 9: 5532, 1: 2451}
Updated value counts:
{2: 9543, 3: 5532, 1: 2451}

Processing column: DISTUR_RENAL
Initial value counts:
{2: 9126, 9: 5951, 1: 2443, 4: 6}
Updated value counts:
{2: 9126, 3: 5957, 1: 2443}

Processing column: SINAL_FAGET
Initial value counts:
{2: 8883, 9: 6294, 1: 2345, 4: 4}
Updated value counts:
{2: 8883, 3: 6298, 1: 2345}

Processing column: DOR_ABDOMINAL
Initial value counts:
{9: 7545, 1: 5977, 2: 3998, 4: 6}
Updated value counts:
{3: 7551, 1: 5977, 2: 3998}

Processing column: RESULT_IGM
Initial value counts:
{2: 8352, 4: 7149, 1: 1861, 3: 164}
Updated value counts:
{2: 8352, 3: 7313, 1: 1861}

Processing column: RESULT_PCR
Initial value counts:
{4: 11008, 2: 4

In [ ]:
base['SEXO'].value_counts()

,count
SEXO,
1,12047
0,5479


In [ ]:
base['CLAS_FINAL'].value_counts()

,count
CLAS_FINAL,
2,13091
1,2349
3,2086


In [ ]:
base['VACINA_FA'].value_counts()

,count
VACINA_FA,
2,7336
1,5899
3,4291


In [ ]:
base['SINT_HEMORRAGICO'].value_counts()

,count
SINT_HEMORRAGICO,
2,9543
3,5532
1,2451


In [ ]:
base['DISTUR_RENAL'].value_counts()

,count
DISTUR_RENAL,
2,9126
3,5957
1,2443


In [ ]:
base['SINAL_FAGET'].value_counts()

,count
SINAL_FAGET,
2,8883
3,6298
1,2345


In [ ]:
base['DOR_ABDOMINAL'].value_counts()

,count
DOR_ABDOMINAL,
3,7551
1,5977
2,3998


In [ ]:
base['RESULT_IGM'].value_counts()

,count
RESULT_IGM,
2,8352
3,7313
1,1861


In [ ]:
base['RESULT_PCR'].value_counts()

,count
RESULT_PCR,
3,11032
2,4712
1,1782


In [ ]:
base['HISTOPATOLOGIA'].value_counts()

,count
HISTOPATOLOGIA,
3,17351
2,145
1,30


In [ ]:
base['R_ISOVIRAL'].value_counts()

,count
R_ISOVIRAL,
3,16434
2,923
1,169


In [ ]:
base['CRIT_CONF'].value_counts()

,count
CRIT_CONF,
3,12122
1,4774
2,630


In [ ]:
base['OBITO'].value_counts()

OBITO
0    15430
1     2096
Name: count, dtype: int64


## Preparação base

Nessa etapa será feito um recorte na base para que ela fique na proporção 1500:1000 para ÓBITO 0:1, pois havia um desbalanceamento muito grande que atrapalharia o desempenho do modelo

Diminuindo ÓBITO = 0

In [ ]:
initial_rows = base.shape[0]
print(f"Número inicial de linhas no DataFrame 'base': {initial_rows}")

# Conte o número de linhas onde 'OBITO' é igual a 0
num_obito_0 = base[base['OBITO'] == 0].shape[0]
print(f"Número de linhas com 'OBITO' = 0: {num_obito_0}")

# Defina o número alvo de remoções
target_removals = 13930

# Verifique se é possível realizar as exclusões solicitadas
if num_obito_0 < target_removals:
    print(f"Aviso: O número de linhas com 'OBITO' = 0 ({num_obito_0}) é menor que as exclusões solicitadas ({target_removals}).")
    print(f"Serão removidas no máximo {num_obito_0} linhas com 'OBITO' = 0.")
    target_removals = num_obito_0
else:
    print(f"É possível remover {target_removals} linhas com 'OBITO' = 0 conforme solicitado.")

# Inicialize um conjunto para armazenar os índices das linhas a serem removidas
indices_to_remove = set()
print("Conjunto 'indices_to_remove' inicializado com sucesso.")

Número inicial de linhas no DataFrame 'base': 17526
Número de linhas com 'OBITO' = 0: 15430
É possível remover 13930 linhas com 'OBITO' = 0 conforme solicitado.
Conjunto 'indices_to_remove' inicializado com sucesso.


In [ ]:
candidate_indices = {}

target_columns = [
    'VACINA_FA', 'SINT_HEMORRAGICO', 'DISTUR_RENAL', 'SINAL_FAGET',
    'DOR_ABDOMINAL', 'RESULT_IGM', 'RESULT_PCR', 'HISTOPATOLOGIA',
    'IMUNOHISTOQUIMICA', 'R_ISOVIRAL', 'CLAS_FINAL', 'CRIT_CONF'
]

print("Coletando índices de linhas candidatas para remoção...")
for col in target_columns:
    candidates = base[(base[col] == 3) & (base['OBITO'] == 0)].index.tolist()
    # Using list(reversed(...)) to allow pop() to remove from the logical 'front' when treated as a stack
    candidate_indices[col] = list(reversed(candidates))
    print(f"Coluna '{col}': {len(candidates)} candidatos encontrados.")

print("'candidate_indices' populado com os índices candidatos para cada coluna.")

Coletando índices de linhas candidatas para remoção...
Coluna 'VACINA_FA': 3596 candidatos encontrados.
Coluna 'SINT_HEMORRAGICO': 4793 candidatos encontrados.
Coluna 'DISTUR_RENAL': 5175 candidatos encontrados.
Coluna 'SINAL_FAGET': 5380 candidatos encontrados.
Coluna 'DOR_ABDOMINAL': 6482 candidatos encontrados.
Coluna 'RESULT_IGM': 6444 candidatos encontrados.
Coluna 'RESULT_PCR': 9860 candidatos encontrados.
Coluna 'HISTOPATOLOGIA': 15312 candidatos encontrados.
Coluna 'IMUNOHISTOQUIMICA': 15321 candidatos encontrados.
Coluna 'R_ISOVIRAL': 14417 candidatos encontrados.
Coluna 'CLAS_FINAL': 1929 candidatos encontrados.
Coluna 'CRIT_CONF': 10852 candidatos encontrados.
'candidate_indices' populado com os índices candidatos para cada coluna.


In [ ]:
print("Iniciando processo de remoção alternada...")
active_columns = list(target_columns)
removed_from_columns = {col: 0 for col in target_columns}
iteration = 0
ignored_columns_during_round_robin = set()

while len(indices_to_remove) < target_removals and active_columns:
    columns_to_remove_from_active = []

    for col in list(active_columns): # Iterate over a copy to allow modification of active_columns
        if len(indices_to_remove) >= target_removals:
            break

        if candidate_indices[col]:
            idx = candidate_indices[col].pop() # Get an index from the list
            indices_to_remove.add(idx)
            removed_from_columns[col] += 1
        else:
            if col not in ignored_columns_during_round_robin:
                print(f"A coluna '{col}' ficou sem candidatos elegíveis.")
                ignored_columns_during_round_robin.add(col)
            columns_to_remove_from_active.append(col)

    # Remove empty columns from the list of active columns
    for col_to_remove in columns_to_remove_from_active:
        if col_to_remove in active_columns:
            active_columns.remove(col_to_remove)

    iteration += 1

    if not active_columns and len(indices_to_remove) < target_removals:
        print("Todas as colunas ativas esgotaram seus candidatos. Não é possível remover mais linhas.")
        break

print(f"\nTotal de linhas selecionadas para remoção: {len(indices_to_remove)}")
if len(indices_to_remove) == target_removals:
    print(f"O número alvo de {target_removals} exclusões foi atingido.")
else:
    print(f"Não foi possível atingir o número alvo de {target_removals} exclusões. Foram removidas {len(indices_to_remove)} linhas.")


Iniciando processo de remoção alternada...
A coluna 'CLAS_FINAL' ficou sem candidatos elegíveis.
A coluna 'VACINA_FA' ficou sem candidatos elegíveis.
A coluna 'SINT_HEMORRAGICO' ficou sem candidatos elegíveis.
A coluna 'DISTUR_RENAL' ficou sem candidatos elegíveis.

Total de linhas selecionadas para remoção: 13930
O número alvo de 13930 exclusões foi atingido.


In [ ]:
print("\nRemovendo as linhas do DataFrame...")
base.drop(index=list(indices_to_remove), inplace=True)
final_rows = base.shape[0]

# Registrar Logs e Comentários Finais
print("\n--- Resumo da Operação ---")
print(f"Número inicial de linhas no DataFrame: {initial_rows}")
print(f"Número exato de linhas removidas: {len(indices_to_remove)}")
print(f"Número final de linhas no DataFrame: {final_rows}")

if initial_rows - len(indices_to_remove) == final_rows:
    print("A contagem de linhas final está consistente com as remoções.")
else:
    print("Atenção: A contagem de linhas final pode não estar consistente.")

print("\nNúmero de linhas removidas por coluna (se aplicável):")
for col, count in removed_from_columns.items():
    if count > 0:
        print(f"- '{col}': {count} linhas contribuíram para a remoção.")

if ignored_columns_during_round_robin:
    print(f"\nAs seguintes colunas foram ignoradas durante o round-robin por ficarem sem candidatos antes do final: {', '.join(ignored_columns_during_round_robin)}")

print("\nOperação concluída.")


Removendo as linhas do DataFrame...

--- Resumo da Operação ---
Número inicial de linhas no DataFrame: 17526
Número exato de linhas removidas: 13930
Número final de linhas no DataFrame: 3596
A contagem de linhas final está consistente com as remoções.

Número de linhas removidas por coluna (se aplicável):
- 'VACINA_FA': 3596 linhas contribuíram para a remoção.
- 'SINT_HEMORRAGICO': 4793 linhas contribuíram para a remoção.
- 'DISTUR_RENAL': 5175 linhas contribuíram para a remoção.
- 'SINAL_FAGET': 5335 linhas contribuíram para a remoção.
- 'DOR_ABDOMINAL': 5335 linhas contribuíram para a remoção.
- 'RESULT_IGM': 5335 linhas contribuíram para a remoção.
- 'RESULT_PCR': 5335 linhas contribuíram para a remoção.
- 'HISTOPATOLOGIA': 5335 linhas contribuíram para a remoção.
- 'IMUNOHISTOQUIMICA': 5335 linhas contribuíram para a remoção.
- 'R_ISOVIRAL': 5335 linhas contribuíram para a remoção.
- 'CLAS_FINAL': 1929 linhas contribuíram para a remoção.
- 'CRIT_CONF': 5335 linhas contribuíram par

In [ ]:
base['OBITO'].value_counts()

,count
OBITO,
1,2096
0,1500


Diminuindo ÓBITO = 1

In [ ]:
import pandas as pd

# Preparar e Verificar Viabilidade
initial_rows = base.shape[0]
print(f"Número inicial de linhas no DataFrame 'base': {initial_rows}")

# Conte o número de linhas onde 'OBITO' é igual a 1
num_obito_1 = base[base['OBITO'] == 1].shape[0]
print(f"Número de linhas com 'OBITO' = 1: {num_obito_1}")

# Define o número alvo de remoções
target_removals = 1096

# Verifique se é possível realizar as exclusões solicitadas
if num_obito_1 < target_removals:
    print(f"Aviso: O número de linhas com 'OBITO' = 1 ({num_obito_1}) é menor que as exclusões solicitadas ({target_removals}).")
    print(f"Serão removidas no máximo {num_obito_1} linhas com 'OBITO' = 1.")
    target_removals = num_obito_1
else:
    print(f"É possível remover {target_removals} linhas com 'OBITO' = 1 conforme solicitado.")

# Inicialize um conjunto para armazenar os índices das linhas a serem removidas
indices_to_remove = set()
print("Conjunto 'indices_to_remove' inicializado com sucesso.")

Número inicial de linhas no DataFrame 'base': 3596
Número de linhas com 'OBITO' = 1: 2096
É possível remover 1096 linhas com 'OBITO' = 1 conforme solicitado.
Conjunto 'indices_to_remove' inicializado com sucesso.


In [ ]:
candidate_indices = {}

# Colunas para procurar o valor '3'
target_columns = [
    'VACINA_FA', 'SINT_HEMORRAGICO', 'DISTUR_RENAL', 'SINAL_FAGET',
    'DOR_ABDOMINAL', 'RESULT_IGM', 'RESULT_PCR', 'HISTOPATOLOGIA',
    'IMUNOHISTOQUIMICA', 'R_ISOVIRAL', 'CLAS_FINAL', 'CRIT_CONF'
]

print("\nColetando índices de linhas candidatas para remoção...")
for col in target_columns:
    # Seleciona índices onde a coluna tem valor '3' E 'OBITO' é '1'
    # Usamos .index.tolist() para obter uma lista de índices
    # e list(reversed(...)) para que o .pop() remova do início da lista,
    # agindo como uma fila LIFO, ou seja, pegando os indices na ordem que foram encontrados
    candidates = base[(base[col] == 3) & (base['OBITO'] == 1)].index.tolist()
    candidate_indices[col] = list(reversed(candidates))
    print(f"Coluna '{col}': {len(candidates)} candidatos encontrados.")

print("'candidate_indices' populado com os índices candidatos para cada coluna.")


Coletando índices de linhas candidatas para remoção...
Coluna 'VACINA_FA': 695 candidatos encontrados.
Coluna 'SINT_HEMORRAGICO': 739 candidatos encontrados.
Coluna 'DISTUR_RENAL': 782 candidatos encontrados.
Coluna 'SINAL_FAGET': 918 candidatos encontrados.
Coluna 'DOR_ABDOMINAL': 1069 candidatos encontrados.
Coluna 'RESULT_IGM': 869 candidatos encontrados.
Coluna 'RESULT_PCR': 1172 candidatos encontrados.
Coluna 'HISTOPATOLOGIA': 2039 candidatos encontrados.
Coluna 'IMUNOHISTOQUIMICA': 1975 candidatos encontrados.
Coluna 'R_ISOVIRAL': 2017 candidatos encontrados.
Coluna 'CLAS_FINAL': 157 candidatos encontrados.
Coluna 'CRIT_CONF': 1270 candidatos encontrados.
'candidate_indices' populado com os índices candidatos para cada coluna.


In [ ]:
print("Iniciando processo de remoção alternada...")
active_columns = list(target_columns)
removed_from_columns = {col: 0 for col in target_columns}
iteration = 0
ignored_columns_during_round_robin = set()

while len(indices_to_remove) < target_removals and active_columns:
    columns_to_remove_from_active = []

    for col in list(active_columns): # Iterar sobre uma cópia para permitir modificação de active_columns
        if len(indices_to_remove) >= target_removals:
            break

        if candidate_indices[col]:
            idx = candidate_indices[col].pop() # Pega um índice da lista
            indices_to_remove.add(idx)
            removed_from_columns[col] += 1
        else:
            if col not in ignored_columns_during_round_robin:
                print(f"A coluna '{col}' ficou sem candidatos elegíveis.")
                ignored_columns_during_round_robin.add(col)
            columns_to_remove_from_active.append(col)

    # Remover colunas vazias da lista de colunas ativas
    for col_to_remove in columns_to_remove_from_active:
        if col_to_remove in active_columns:
            active_columns.remove(col_to_remove)

    iteration += 1

    if not active_columns and len(indices_to_remove) < target_removals:
        print("Todas as colunas ativas esgotaram seus candidatos. Não é possível remover mais linhas.")
        break

print(f"\nTotal de linhas selecionadas para remoção: {len(indices_to_remove)}")
if len(indices_to_remove) == target_removals:
    print(f"O número alvo de {target_removals} exclusões foi atingido.")
else:
    print(f"Não foi possível atingir o número alvo de {target_removals} exclusões. Foram removidas {len(indices_to_remove)} linhas.")

Iniciando processo de remoção alternada...
A coluna 'CLAS_FINAL' ficou sem candidatos elegíveis.

Total de linhas selecionadas para remoção: 1096
O número alvo de 1096 exclusões foi atingido.


In [ ]:
print("\nRemovendo as linhas do DataFrame...")
base.drop(index=list(indices_to_remove), inplace=True)
final_rows = base.shape[0]

# Registrar Logs e Comentários Finais
print("\n--- Resumo da Operação ---")
print(f"Número inicial de linhas no DataFrame: {initial_rows}")
print(f"Número exato de linhas removidas: {len(indices_to_remove)}")
print(f"Número final de linhas no DataFrame: {final_rows}")

if initial_rows - len(indices_to_remove) == final_rows:
    print("A contagem de linhas final está consistente com as remoções.")
else:
    print("Atenção: A contagem de linhas final pode não estar consistente.")

print("\nNúmero de linhas removidas por coluna (se aplicável):")
for col, count in removed_from_columns.items():
    if count > 0:
        print(f"- '{col}': {count} linhas contribuíram para a remoção.")

if ignored_columns_during_round_robin:
    print(f"\nAs seguintes colunas foram ignoradas durante o round-robin por ficarem sem candidatos antes do final: {', '.join(ignored_columns_during_round_robin)}")

print("\nOperação concluída.")


Removendo as linhas do DataFrame...

--- Resumo da Operação ---
Número inicial de linhas no DataFrame: 3596
Número exato de linhas removidas: 1096
Número final de linhas no DataFrame: 2500
A contagem de linhas final está consistente com as remoções.

Número de linhas removidas por coluna (se aplicável):
- 'VACINA_FA': 266 linhas contribuíram para a remoção.
- 'SINT_HEMORRAGICO': 266 linhas contribuíram para a remoção.
- 'DISTUR_RENAL': 266 linhas contribuíram para a remoção.
- 'SINAL_FAGET': 266 linhas contribuíram para a remoção.
- 'DOR_ABDOMINAL': 266 linhas contribuíram para a remoção.
- 'RESULT_IGM': 266 linhas contribuíram para a remoção.
- 'RESULT_PCR': 266 linhas contribuíram para a remoção.
- 'HISTOPATOLOGIA': 266 linhas contribuíram para a remoção.
- 'IMUNOHISTOQUIMICA': 266 linhas contribuíram para a remoção.
- 'R_ISOVIRAL': 266 linhas contribuíram para a remoção.
- 'CLAS_FINAL': 157 linhas contribuíram para a remoção.
- 'CRIT_CONF': 266 linhas contribuíram para a remoção.



In [ ]:
base['OBITO'].value_counts()

,count
OBITO,
0,1500
1,1000


Análise de algumas colunas

In [ ]:
print(base.columns.tolist())

['UF_NOTIF', 'REG_NOTIF', 'IDADE', 'SEXO', 'VACINA_FA', 'SINT_HEMORRAGICO', 'DISTUR_RENAL', 'SINAL_FAGET', 'DOR_ABDOMINAL', 'HOSPITALIZACAO', 'RESULT_IGM', 'RESULT_PCR', 'HISTOPATOLOGIA', 'IMUNOHISTOQUIMICA', 'R_ISOVIRAL', 'CLAS_FINAL', 'CRIT_CONF', 'EVOLUCAO', 'AUTOCTONE', 'PERI_SAZONAL', 'DIAS_ATE_NOTIF', 'OBITO', 'DIAS_ATE_INTERNACAO', 'INTERNACAO']


In [ ]:
base.describe(include='all').T

Removendo linhas EVOLUCAO, pois está fortemente relacionada com a classe alvo, CLAS_FINAL, pelo mesmo motivo e DIAS_ATE_INTERNACAO, pela ausencia de valores

In [ ]:
columns_to_drop = ['EVOLUCAO', 'CLAS_FINAL', 'DIAS_ATE_INTERNACAO']
base = base.drop(columns=columns_to_drop, errors='ignore')

print("Colunas removidas com sucesso.")
print("\nColunas restantes no DataFrame:")
print(base.columns.tolist())

Colunas removidas com sucesso.

Colunas restantes no DataFrame:
['UF_NOTIF', 'IDADE', 'SEXO', 'VACINA_FA', 'SINT_HEMORRAGICO', 'DISTUR_RENAL', 'SINAL_FAGET', 'DOR_ABDOMINAL', 'RESULT_IGM', 'RESULT_PCR', 'HISTOPATOLOGIA', 'IMUNOHISTOQUIMICA', 'R_ISOVIRAL', 'CRIT_CONF', 'PERI_SAZONAL', 'DIAS_ATE_NOTIF', 'OBITO', 'INTERNACAO']


In [ ]:
base.to_excel('basePreProcessamento2500Instancias.xlsx', index=False)
print("DataFrame exportado com sucesso como 'basePreProcessamento2500Instancias.xlsx'")

DataFrame exportado com sucesso como 'basePreProcessamento2500Instancias.xlsx'
